In [1]:
import numpy as np
import pandas as pd
import sklearn

In [32]:
# training data
train_docs = pd.read_csv('example_train.csv')
train_docs

,Document,Class
0,Upgrad is a great educational institution.,education
1,Educational greatness depends on ethics,education
2,A story of great ethics and educational greatness,education
3,Sholey is a great cinema,cinema
4,good movie depends on good story,cinema


In [33]:
# Convert label to a numeric variable
train_docs['Class'] = train_docs['Class'].map({'cinema': 0, 'education': 1})

In [34]:
train_docs

,Document,Class
0,Upgrad is a great educational institution.,1
1,Educational greatness depends on ethics,1
2,A story of great ethics and educational greatness,1
3,Sholey is a great cinema,0
4,good movie depends on good story,0


In [35]:
# convert the DF into a numpy array
train_array = train_docs.values

# Split X and Y
X_train = train_array[:,0]
y_train = train_array[:,1]
y_train = y_train.astype('int') # sklearn needs y as int

print("X_train")
print(X_train)
print("y_train")
print(y_train)

X_train
['Upgrad is a great educational institution.'
 'Educational greatness depends on ethics'
 'A story of great ethics and educational greatness'
 'Sholey is a great cinema' 'good movie depends on good story']
y_train
[1 1 1 0 0]


In [12]:
# Convert string into Bag of Words

In [36]:
# create  an object of countVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [37]:
vec = CountVectorizer()

In [38]:
# fit vectorizer on training data
vec.fit(X_train)
vec.vocabulary_

{'upgrad': 15,
 'is': 9,
 'great': 6,
 'educational': 3,
 'institution': 8,
 'greatness': 7,
 'depends': 2,
 'on': 12,
 'ethics': 4,
 'story': 14,
 'of': 11,
 'and': 0,
 'sholey': 13,
 'cinema': 1,
 'good': 5,
 'movie': 10}

In [39]:
# remove stop words
vec = CountVectorizer(stop_words='english')
vec.fit(X_train)
vec.vocabulary_

{'upgrad': 11,
 'great': 5,
 'educational': 2,
 'institution': 7,
 'greatness': 6,
 'depends': 1,
 'ethics': 3,
 'story': 10,
 'sholey': 9,
 'cinema': 0,
 'good': 4,
 'movie': 8}

In [40]:
print(vec.get_feature_names())
print(len(vec.get_feature_names()))

['cinema', 'depends', 'educational', 'ethics', 'good', 'great', 'greatness', 'institution', 'movie', 'sholey', 'story', 'upgrad']
12


In [41]:
# compress sparse matrix - sparse because not all records will all words and in BoW representations lot of words count will be zeroes
X_transformed = vec.transform(X_train)
X_transformed

<5x12 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [42]:
print(X_transformed)
# this says for document 0 (first document) feature number 2 occurs once
# for document number 4 feature number 4 occurs twice

  (0, 2)	1
  (0, 5)	1
  (0, 7)	1
  (0, 11)	1
  (1, 1)	1
  (1, 2)	1
  (1, 3)	1
  (1, 6)	1
  (2, 2)	1
  (2, 3)	1
  (2, 5)	1
  (2, 6)	1
  (2, 10)	1
  (3, 0)	1
  (3, 5)	1
  (3, 9)	1
  (4, 1)	1
  (4, 4)	2
  (4, 8)	1
  (4, 10)	1


In [43]:
# converting transformed matrix into an array
X_transformed.toarray()

array([[0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0]], dtype=int64)

In [44]:
# just for our understanding convert this into DataFrame
pd.DataFrame(X_transformed.toarray(), columns=vec.get_feature_names())

,cinema,depends,educational,ethics,good,great,greatness,institution,movie,sholey,story,upgrad
0,0,0,1,0,0,1,0,1,0,0,0,1
1,0,1,1,1,0,0,1,0,0,0,0,0
2,0,0,1,1,0,1,1,0,0,0,1,0
3,1,0,0,0,0,1,0,0,0,1,0,0
4,0,1,0,0,2,0,0,0,1,0,1,0


In [45]:
# test data
test_docs = pd.read_csv('example_test.csv')
test_docs

,Document,Class
0,very good educational institution,education


In [46]:
# convery label to a numerical variable
test_docs['Class'] = test_docs['Class'].map({'cinema': 0, 'education': 1})
test_docs

,Document,Class
0,very good educational institution,1


In [47]:
# convert the DF into a numpy array
test_array = test_docs.values

# Split X and Y
X_test = test_array[:,0]
y_test = test_array[:,1]
y_test = y_test.astype('int') # sklearn needs y as int

print("X_test")
print(X_test)
print("y_test")
print(y_test)

X_test
['very good educational institution']
y_test
[1]


In [48]:
X_test_transformed = vec.transform(X_test)
X_test_transformed

<1x12 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [52]:
from sklearn.naive_bayes import MultinomialNB

# instantiate NB
mnb = MultinomialNB()

# fit model on training data
mnb.fit(X_transformed, y_train)

# predicting probabilities
proba = mnb.predict_proba(X_test_transformed)

In [55]:
# probability of cinema
print(proba[:,0])

# probability of education
print(proba[:,1])

[0.32808399]
[0.67191601]
